In [2]:
import pandas as pd 
import numpy as np 
import seaborn as sns
from sklearn.impute import SimpleImputer
from deep_translator import GoogleTranslator
import re
from bs4 import BeautifulSoup
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords

In [3]:
articles = pd.read_csv('./../art_clean_wt_popul_authoroccurences.csv')

In [4]:
articles.head(2)

,nzz_id,author,catchline,content,content_length,department,lead_text,pub_date,title,popularity,author_occurences,author_popularity
0,ld.149648,Claudia Gabriel,Obligationenfonds mit fixer Laufzeit,Obligationenfonds mit fixer Laufzeit Es gi...,704,Finanzen,Die Idee ist gut: Statt einer einzigen Obligat...,2017-03-09 08:01:21.000,Es gibt noch interessante Varianten,0.009009,57,0.157593
1,1.18145900,Unknown,Unknown,NaN,0,Panorama,Zum Auftakt der Fashion Week in New York zeige...,2017-04-11 14:00:29.473,Fashion Week New York,0.000000,0,0.000000


In [5]:
# dropping columns not needed for content-based model
articles = articles.drop(['popularity','author_occurences','author_popularity'], axis=1)

In [6]:
# converting NaN to empty string
articles = articles.replace(np.nan, "")

In [7]:
articles.content[1]

''

### Cleaning

In [8]:
# already done
# content intro cleaning (html + hard space)
def clean_html(data):
    html_clean = re.compile(r'<.*?>')
    data = html_clean.sub(r' ',data)
    return data

def clean_data(row):
    row = str(BeautifulSoup(row))
    row = clean_html(row).replace(u'\xa0', u' ')
    return row

In [9]:
# further cleaning
def clean_url(data):
    url_clean = re.compile(r"https://\S+|www\.\S+")
    data = url_clean.sub(r'',data)
    return data

# Modify to lower - to reduce number of words and exclude duplicates differing by letter case
def clean_to_lower(text):
    return text.lower()

# Remove punctuations - all special chars 
def clean_special_char(data): # and numbers - also not very informative without context
    # data = re.sub('\[[^]]*\]', ' ', data)
    # data = re.sub('[^a-zA-Z]', ' ', data) # - german letters!!!
    data = re.sub(r'[^\w\s]',' ',data)
    return data

# Match all digits in the string and replace them by ' '
def clean_numbers(data): # to decide whether to use or not !!!
    data = re.sub(r'[0-9]', ' ', data)
    return data

In [10]:
# cleaning pipeline
def clean_data_2(data):
    data = data.apply(clean_url).apply(clean_to_lower).apply(clean_special_char).apply(clean_numbers)
    return data

In [11]:
# applying cleaning
articles_nlp = articles
articles_nlp['content'] = clean_data_2(articles_nlp['content'])

In [12]:
articles_nlp.content[2]

'   e banking ausfall  postfinance kämpft mit dem system     seit sonntag funktioniert das e banking der postfinance nicht mehr  das institut spricht von einem internen systemproblem  fachleute argumentieren  es könnte sich auch um einen  überlastungsangriff  handeln    gru   sda  viele bankkunden nutzen jeweils das wochenende  um ihre zahlungen zu erledigen  bei der postfinance war dies am vergangenen sonntag aber nicht möglich  seit sonntagvormittag konnten die     mio  kunden der post tochter über e banking nicht auf ihre konten zugreifen  ein sprecher des unternehmens bestätigte eine entsprechende online meldung des zürcher  tages anzeigers    die kundendaten seien von den technischen problemen laut dem ngeldhaus aber nicht gefährdet   umfang der panne wird nicht publiziert  betroffen sind alle postfinance kunde  es gebe keine kundengruppen oder regionen  die spezifisch betroffen seien  sagt postfinance sprechernjohannes möri  wie viele kunden erfolglos versuchten auf ihr konto zuz

### Content_length

In [13]:
# word count for each article - includes stopwords
splitted = [x.split() for x in articles_nlp['content']] # word count
articles_nlp['content_len'] = [len(x) for x in splitted]

In [14]:
articles_nlp.head()

,nzz_id,author,catchline,content,content_length,department,lead_text,pub_date,title,content_len
0,ld.149648,Claudia Gabriel,Obligationenfonds mit fixer Laufzeit,obligationenfonds mit fixer laufzeit es gi...,704,Finanzen,Die Idee ist gut: Statt einer einzigen Obligat...,2017-03-09 08:01:21.000,Es gibt noch interessante Varianten,718
1,1.18145900,Unknown,Unknown,,0,Panorama,Zum Auftakt der Fashion Week in New York zeige...,2017-04-11 14:00:29.473,Fashion Week New York,0
2,ld.138769,Unknown,E-Banking-Ausfall,e banking ausfall postfinance kämpft mit d...,523,Finanzen,Seit Sonntag funktioniert das E-Banking der Po...,2017-01-09 13:55:00.000,Postfinance kämpft mit dem System,525
3,ld.143700,Unknown,Terror in Frankreich,terror in frankreich louvre nach macheten ...,180,International,Einen Tag nach dem Angriff auf Soldaten beim P...,2017-02-04 12:50:25.000,Louvre nach Macheten-Angriff wieder geöffnet,181
4,ld.149385,Unknown,Unglück in Panama,unglück in panama bus prallt gegen eine ma...,117,Panorama,Bei einem Busunglück in Panama sind 17 Persone...,2017-03-06 07:31:21.000,Bus prallt gegen eine Mauer und stürzt in Fluss,110


### Tokens

In [15]:
# stopwords
# stopwords.words('german')

In [16]:
def tokenize_and_remove_stopwords(row):
    token = word_tokenize(row) 
    filtered_token = []
    for word in token:
        if word not in stopwords.words('german'):
            filtered_token.append(word)
    return filtered_token

In [18]:
# articles_nlp.loc[0:2,'content'].apply(tokenize_and_remove_stopwords)[2]

In [19]:
%%time
tokens = articles_nlp['content'].apply(tokenize_and_remove_stopwords)

Wall time: 37min 21s


In [22]:
tokens.to_csv("tokens.csv", encoding="utf-8", index=False)

In [23]:
# tok_test = pd.read_csv('tokens.csv')

In [28]:
articles_nlp.to_csv('articles_nlp.csv', index=False, encoding='utf-8')

### Wektoryzacja

In [ ]:
stopwords_list = stopwords.words('german') # are there other languages in text

vectorizer = TfidfVectorizer(analyzer='word',
                     ngram_range=(1, 2), # bi - gram (?)
                     min_df=0.003,
                     max_df=0.5,
                     max_features=5000,
                     stop_words=stopwords_list)

item_ids = articles_nlp['nzz_id'].tolist()
tfidf_matrix = vectorizer.fit_transform(articles_nlp['content'])
tfidf_feature_names = vectorizer.get_feature_names()
tfidf_matrix